# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


In [2]:
import os
print("API Key:", os.getenv("ALPHAVANTAGE_API_KEY"))

API Key: IHHY6ORZJGIOCFN2


## Helpers (use or modify)

In [3]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [4]:
!pip install yfinance --upgrade --no-cache-dir

In [5]:
import os
import yfinance as yf
import pandas as pd
from datetime import datetime

ticker = "AAPL"
df_api = yf.download(ticker, period="1mo", interval="1d")

# Flatten MultiIndex columns if needed
if isinstance(df_api.columns, pd.MultiIndex):
    df_api.columns = [col[0] for col in df_api.columns]

df_api.reset_index(inplace=True)

# Parse dtypes
if "Date" in df_api.columns:
    df_api["Date"] = pd.to_datetime(df_api["Date"], errors="coerce")

for col in df_api.columns:
    if col not in ["Date"]:
        df_api[col] = pd.to_numeric(df_api[col], errors="coerce")

# Validation
print("Shape:", df_api.shape)
print("Columns:", df_api.columns)
print("NA counts:\n", df_api.isna().sum())

# Save
os.makedirs("data/raw", exist_ok=True)
fname_api = f"data/raw/api_yfinance_{ticker}_{datetime.now().strftime('%Y%m%d-%H%M')}.csv"
df_api.to_csv(fname_api, index=False)
print("Saved:", fname_api)

/var/folders/78/k4tp9c153gd34_2034rhhqfw0000gn/T/ipykernel_60532/1733745642.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(ticker, period="1mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

Shape: (23, 6)
Columns: Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
NA counts:
 Date      0
Close     0
High      0
Low       0
Open      0
Volume    0
dtype: int64
Saved: data/raw/api_yfinance_AAPL_20250820-1356.csv


In [6]:
!pip install lxml

In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# Find the first table
table = soup.find("table", {"id": "constituents"})

# Parse into DataFrame
df_scrape = pd.read_html(str(table))[0]

# Validation
print("Columns:", df_scrape.columns)
print("Shape:", df_scrape.shape)
print("NA counts:\n", df_scrape.isna().sum())

# Save
fname_scrape = f"data/raw/scrape_wikipedia_sp500_{datetime.now().strftime('%Y%m%d-%H%M')}.csv"
df_scrape.to_csv(fname_scrape, index=False)
print("Saved:", fname_scrape)

Columns: Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')
Shape: (503, 8)
NA counts:
 Symbol                   0
Security                 0
GICS Sector              0
GICS Sub-Industry        0
Headquarters Location    0
Date added               0
CIK                      0
Founded                  0
dtype: int64
Saved: data/raw/scrape_wikipedia_sp500_20250820-1356.csv


/var/folders/78/k4tp9c153gd34_2034rhhqfw0000gn/T/ipykernel_60532/1895318161.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_scrape = pd.read_html(str(table))[0]


In [8]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

IndexError: list index out of range

Edited Code Snippet becuase of ALPHAVANTAGE_API key

In [9]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

df_api = None

if USE_ALPHA:
    try:
        url = 'https://www.alphavantage.co/query'
        params = {
            'function': 'TIME_SERIES_DAILY_ADJUSTED',
            'symbol': SYMBOL,
            'outputsize': 'compact',
            'apikey': os.getenv('ALPHAVANTAGE_API_KEY')
        }
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()

        key_candidates = [k for k in js if 'Time Series' in k]
        if key_candidates:
            key = key_candidates[0]
            df_api = (
                pd.DataFrame(js[key])
                .T.reset_index()
                .rename(columns={'index': 'date', '5. adjusted close': 'adj_close'})
                [['date','adj_close']]
            )
            df_api['date'] = pd.to_datetime(df_api['date'])
            df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
        else:
            print("Alpha Vantage response missing data, falling back to yfinance.")
            USE_ALPHA = False
    except Exception as e:
        print("Alpha Vantage failed:", e)
        USE_ALPHA = False

if not USE_ALPHA or df_api is None:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d')

    # Flatten MultiIndex columns if needed
    if isinstance(df_api.columns, pd.MultiIndex):
        df_api.columns = [c[0] for c in df_api.columns]

    df_api = df_api.reset_index()

    # Select adjusted close if available, else close
    if 'Adj Close' in df_api.columns:
        df_api = df_api[['Date','Adj Close']]
    else:
        df_api = df_api[['Date','Close']].rename(columns={'Close':'Adj Close'})

    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close'])
v_api

/var/folders/78/k4tp9c153gd34_2034rhhqfw0000gn/T/ipykernel_60532/1287104549.py:39: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d')
[*********************100%***********************]  1 of 1 completed

Alpha Vantage response missing data, falling back to yfinance.


{'missing': [], 'shape': (64, 2), 'na_total': 0}

In [10]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-yfinance_symbol-AAPL_20250820-135638.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [11]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (880, 8), 'na_total': 758}

In [12]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250820-135641.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.